In [1]:
import pandas as pd
import numpy as py
import json
import re
import jieba
import os
from pathlib import Path

In [126]:
youtuber_list = pd.read_csv('youtuber_list.csv')
youtuber_list = youtuber_list.rename(columns={"id": "youtuberId"})
youtuber_list.head(3)

,rank,score,name,youtuberId,uploads,subs,views,類別
0,1st,B+,這群人TGOP,UC6FcYHEm7SO1jpu5TKjNXEA,197,3.07M,"714,912,915",NaN
1,3rd,B,阿滴英文,UCeo3JwE3HezUWFdVcehQk9Q,429,2.37M,"204,815,583",NaN
2,4th,B+,蔡阿嘎,UCPwxSX0DYDMQxCvgfeVDv_g,624,2.25M,"505,769,097",NaN


In [ ]:
#把 youtuber list name & json data content 大小 名稱 處理

In [107]:
from collections import defaultdict
from collections import OrderedDict
from langconv import *
import math

def text_preprocess(line):
    line = line.replace('\n','').replace('\r','')
    line = Simplified2Traditional(line)    
    rule = re.compile(u"[^a-zA-Z0-9\u4e00-\u9fa5]")
    line = rule.sub('',line)
    line = re.sub(r"(^|\W)\d+", "", line)
    return line

def Simplified2Traditional(sentence):
    sentence = Converter('zh-hant').convert(sentence)
    return sentence

def get_dictionary(youtube_df):
    big_table=defaultdict(dict)
    i = 0
    for token_list in youtube_df["tokens"]:
        #theArticleDic 存放一篇文章內所有的gram，用來計算DF
        theArticleDic=set()
        #用for迴圈開始切單一篇文章，並且對照big table記錄其TF DF
        for word in token_list:

            if word not in big_table:      
                big_table[word]["tf"]=1
                big_table[word]["df"]=1
                big_table[word]["df_list"]=[i]
                theArticleDic.add(word)
            else:
                big_table[word]["tf"]+=1
                if word not in theArticleDic:
                    big_table[word]["df"]+=1
                    big_table[word]["df_list"].append(i)
                    theArticleDic.add(word)      
        i += 1

    for key in big_table:
        big_table[key]['tf-idf']= big_table[key]["tf"] * math.log(len(youtube_df['tokens'])/big_table[key]["df"])
    table_descend = OrderedDict(sorted(big_table.items(), key=lambda kv: kv[1]['tf-idf'], reverse=True))
    final_table = {k: v for k, v in table_descend.items() if v['tf-idf'] >= 50}
    return final_table, youtube_df['name']

In [127]:
# get rid of XD, 哈哈... words that repeat/ chi sq??

In [4]:
path_to_json = '/Users/student/Desktop/IM final/youtube_comments_data'
files_list = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(files_list)

['UCV1OpUwWJCMiv6gJCRR6yAg.json', 'UC6yxPVu2b52UVJQEwCTzivQ.json', 'UC8MQLq2uFUa-HXHCVjHlc-Q.json', 'UCGbpr46Y8aDyO2s_W_3SfkA.json', 'UC9i2Qgd5lizhVgJrdnxunKw.json', 'UCHm9SiOLG8UoBT8STWY5mVA.json', 'UC3LBFXbWtEBdOOUb8-qJm9Q.json', 'UCRevYqA7N-NrSCGSm5k-KlA.json', 'UCNMIUvGvW35xL294jqY3cwg.json', 'UCpgt8SEyAy5tbr9BzVK8Lsg.json', 'UC0Q-fBheHysYWz9ObSEzMdA.json', 'UCyD3eaCai2yyWZEuczgZ-fw.json', 'UCj_z-Zeqk8LfwVxx0MUdL-Q.json', 'UCp2dhjQXkhY1O40lj15JQzw.json', 'UCJMjp_3XLlQJg6pPfuD4L1Q.json', 'UC4kXdDgmzrycOUk9ONnE03w.json', 'UCHfY_EOzB1i57hYLSw_rYMg.json', 'UCnMlG9P337rFzaJVly5EZDw.json', 'UCCrwpId4G9sRQjLHaUv90YA.json', 'UC7SHHIlKiJs_-ORerKPL1kA.json', 'UCghPiQIi_uyjF1YHKj-FhGw.json', 'UCizhMTp9vKkpdcbqVLdDmdA.json', 'UCflli65jykYa6D0AU8JSuGA.json', 'UCVw4CvsOUuVJpFyW_VPnIBQ.json', 'UCJJDD-Hy76jvUMRG-dpFkcw.json', 'UCRNsqNFhDF8exJpwQuCOXqw.json', 'UCdz-LcN65kBCBrQVGw4ZEmQ.json', 'UCzxN4G3s9uR9ao5_O5DoXmA.json', 'UCqAYPUHgXuzsNlYNIzItfAQ.json', 'UCec6QE03CMLgtf0f1-uDdbw.json', 'UC9qPaWp

In [99]:
#自定義辭典
jieba.load_userdict('user_dict.txt')
#stopwords
with open('stopwords.txt') as f:
    stopWords = [line.strip() for line in f.readlines()]

In [108]:
youtube_path = '/Users/student/Desktop/IM final/youtube_comments_data'
dcard_path = '/Users/student/Desktop/IM final/dcard_data'
#directory = os.fsencode(data_path)

#comments_list: list of combined comments with no punctuations
#token_list: list of word_lists
#word_list: list of words in a video

dictionary_list = []
name_list = []
#youtube 和 dcard 檔名皆為 youtuber_id.json
youtube_df = pd.DataFrame(columns=['Id','comments','tokens','youtuberId',
                                   'rank','score','name','uploads','subs','views','類別'])
export_csv = youtube_df.to_csv ('youtube_df.csv', index = None, header=True)
for filename in files_list[0:2]: 
    Id_list, comments_list, token_list = [], [], []
    youtube_df = pd.DataFrame()
    #讀進每個在 youtube_comments_data 資料夾的檔，存進 data 中
    with open(os.path.join(youtube_path, filename),'r',encoding = 'utf-8') as json_file:
        youtube_data = json.load(json_file)

    for i,video in enumerate(youtube_data):  
        s = ''
        word_list = [] #list of words in a video 
        for comments in video['comments']:
            context = comments['textOriginal']
            context = text_preprocess(context)
            s += context
            #split words
            words = jieba.cut(context, cut_all=False)
            for word in words:
                if len(word) > 1 and word not in stopWords:
                    word_list.append(word)
        if s != '' and video['videoId'] not in Id_list:
            comments_list.append(s)
            Id_list.append(video['videoId'])
            token_list.append(word_list)
            
    dcard_file = Path(os.path.join(dcard_path, filename))
    if dcard_file.is_file():
        with open(dcard_file,'r',encoding = 'utf-8') as json_file:
            dcard_data = json.load(json_file)            
        for data in dcard_data:
            s = ''
            word_list = [] #list of words in a dcard post
            Id = data['id']
            s += text_preprocess(data['title'] + data['excerpt'])
            #combine comments
            if 'comments' in data:
                for comment in data['comments']:
                    if comment['hidden'] == False:
                        c = comment['content']
                        c = text_preprocess(c)
                        s += c
                        #split words
                        words = jieba.cut(c, cut_all=False)
                        for word in words:
                            if len(word) > 1 and word not in stopWords:
                                if 'XD' and '哈哈' and '啊啊' not in word:
                                    word_list.append(word)         
                if s != '':
                    comments_list.append(s)
                    Id_list.append(Id)
                    token_list.append(word_list)
    print(filename,'done')
    
    youtube_df['Id'] = Id_list
    youtube_df['comments'] = comments_list
    youtube_df['tokens'] = token_list
    youtube_df['youtuberId'] = [filename.replace('.json','')] * len(youtube_df)
    youtube_df = pd.merge(youtube_df, youtuber_list, on='youtuberId', how='inner')
    
    with open('youtube_df.csv', 'a') as f:
        (youtube_df).to_csv(f, index = None, header=False)
    
    y_dict, y_name = get_dictionary(youtube_df)
    dictionary_list.append(y_dict)
    name_list.append(y_name)


UCV1OpUwWJCMiv6gJCRR6yAg.json done
UC6yxPVu2b52UVJQEwCTzivQ.json done


In [117]:
temp = pd.read_csv("youtube_df.csv") 
temp.head(1)

,Id,comments,tokens,youtuberId,rank,score,name,uploads,subs,views,類別
0,b_0_Q99h-P0,禮拜六繼續上法國男孩的超台行程帶大家跟我們一起吃熱炒敬請期待酷跟其他youtuber最不同的...,"['禮拜', '六繼續', '法國', '男孩', '超台', '行程', '一起', '吃...",UCV1OpUwWJCMiv6gJCRR6yAg,143rd,B,Ku's dream酷的夢-,119,527K,"35,000,274",NaN


In [123]:
feature_path = '/Users/student/Desktop/IM final/feature_data'
for index, name in enumerate(name_list):
    with open(os.path.join(feature_path, name[0]+'.json'), 'w') as outfile:
        json.dump(dictionary_list[index], outfile)

In [114]:
print(name_list[1][0],list(dictionary_list[1].keys())[1: 25])

老皮 ['好笑', '遊戲', '加油', '嚇到', '封面', '系列', '恭喜', '這遊戲', '喜歡', '影片', '繼續', '期待', '一集', 'XDD', '知道', '啊啊啊', '配音', '皮弟', '好像', '好看', '實況', '好玩', 'XDDD', '結局']


In [116]:
print(name_list[0][0],list(dictionary_list[0].keys())[1: 25])

Ku's dream酷的夢- ['皮蛋', '烤肉', '芋頭', '台語', '酷媽', '中秋', '台灣', '法國', '韓國', '國家', '外國人', '好吃', '高雄', '法國人', '早餐', '韓文', '大謙', '中文', '法式', '旺旺', '奶茶', '日本', '朋友', '好喝']


In [125]:
import datetime
start = datetime.datetime.now()

end = datetime.datetime.now()
print(end-start)

0:00:09.703299


In [93]:
list(final_table.keys())
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
take(10, final_table.items())

[('com',
  {'tf': 132,
   'df': 8,
   'df_list': [1, 17, 39, 65, 67, 85, 100, 123],
   'tf-idf': 469.3059441166026}),
 ('gmail',
  {'tf': 119,
   'df': 8,
   'df_list': [1, 17, 39, 65, 67, 85, 100, 123],
   'tf-idf': 423.0864193172402}),
 ('愛眾量',
  {'tf': 107,
   'df': 8,
   'df_list': [1, 17, 39, 100, 129, 135, 141, 190],
   'tf-idf': 380.42224257936726}),
 ('爺爺',
  {'tf': 78,
   'df': 3,
   'df_list': [122, 125, 182],
   'tf-idf': 353.82183053108895}),
 ('好吃',
  {'tf': 108,
   'df': 29,
   'df_list': [8,
    9,
    10,
    26,
    27,
    53,
    64,
    79,
    84,
    87,
    95,
    98,
    101,
    107,
    113,
    119,
    122,
    128,
    138,
    153,
    165,
    166,
    169,
    189,
    196,
    234,
    235,
    236,
    244],
   'tf-idf': 244.88932750373976}),
 ('快樂',
  {'tf': 104,
   'df': 27,
   'df_list': [45,
    46,
    47,
    61,
    62,
    77,
    85,
    90,
    105,
    108,
    112,
    113,
    122,
    125,
    126,
    138,
    139,
    147,
    153,
   

In [24]:
#feature selection

N = 50
E11 = 20*29/N
E01 = 30*29/50
E10 = 20*21/50
E00 = N-E11-E01-E10
print(E00)
N11 = 3
N10 = 26
N01 = 17
N00 = 4
chisq = ((N11-E11)**2)/E11 + ((N10-E10)**2)/E10 + ((N00-E00)**2)/E00 + ((N01-E01)**2)/E01
chisq

12.6


49.131089217296115

In [86]:
from collections import defaultdict
def ngram(textLists,gram,minFreq): #第一個參數放處理好的文章(LIST檔，utf-8編碼)，第二個參數放字詞的長度單位，第三個參數放至少要幾次以上
    count=0
    big_table=defaultdict(dict)
    for gram in range(2,gram):
        for article in textLists:
            #去掉符號
            article=re.sub("[\s+\.\!\/_,$%^*()+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",article)
            #theArticleDic 存放一篇文章內所有的gram，用來計算DF
            theArticleDic=set()

            #用for迴圈開始切單一篇文章，每次切gram個字元，並且對照big table記錄其TF DF
            for index in range(0,len(article)-gram+1):
                word=article[index:index+gram]

                if word not in big_table:
                    big_table[word]["tf"]=1
                    big_table[word]["df"]=1
                    theArticleDic.add(word)
                else:
                    big_table[word]["tf"]+=1
                    if word not in theArticleDic:
                        big_table[word]["df"]+=1
                        theArticleDic.add(word)
            count+=1
    return big_table ##回傳一個陣列[詞,頻次]

def longTermPriority(text_list, maxTermLength, minFreq):
    longTerms=[]          #長詞
    longTermsFreq=[]      #長詞+次數分配
    
    for i in range(maxTermLength,1,-1):
        
        words_freq = ngram(text_list,i, minFreq)
        #print i
        #print(words_freq)
    
        for word_freq in words_freq:
            longTerms.append(word_freq[0])
            #print word_freq[0]
            longTermsFreq.append(word_freq) 
            #print word_freq
    
    return longTermsFreq
        


In [213]:
from collections import OrderedDict 
from operator import getitem 
import math
import regex as re
        
def strInter(s1, s2):
    out = ""
    for c in s1:
        if c in s2 and not c in out:
            out += c
    return len(out)

def count_overlapping(text, search_for):
    return len(re.findall(text, search_for, overlapped=True))

a = ngram(youtube_df['comments'],7,0)

ordered_dict = OrderedDict(sorted(a.items(), key = lambda x: getitem(x[1], 'tf'),reverse=True)) 
N = len(youtube_df)
for item in ordered_dict.items():
    item[1]['tf-idf'] = item[1]['tf']*math.log(N/item[1]['df'])

#ordered_dict = OrderedDict(sorted(a.items(), key = lambda x: getitem(x[1], 'tf-idf'),reverse=True))
ordered_dict = dict((k, v) for k, v in ordered_dict.items() if v['tf-idf'] >= 5)

In [214]:
ordered_dict

{'家寧': {'tf': 81, 'df': 7, 'tf-idf': 10.816042802586328},
 '量級': {'tf': 70, 'df': 6, 'tf-idf': 20.13774507162466},
 '眾量': {'tf': 62, 'df': 5, 'tf-idf': 29.14022501323561},
 '眾量級': {'tf': 62, 'df': 5, 'tf-idf': 29.14022501323561},
 '我愛': {'tf': 60, 'df': 2, 'tf-idf': 83.17766166719343},
 '愛眾': {'tf': 54, 'df': 3, 'tf-idf': 52.96477966263321},
 '愛眾量': {'tf': 53, 'df': 3, 'tf-idf': 51.983950409621485},
 '愛眾量級': {'tf': 53, 'df': 3, 'tf-idf': 51.983950409621485},
 '我愛眾': {'tf': 50, 'df': 1, 'tf-idf': 103.97207708399179},
 '我愛眾量': {'tf': 49, 'df': 1, 'tf-idf': 101.89263554231195},
 '我愛眾量級': {'tf': 49, 'df': 1, 'tf-idf': 101.89263554231195},
 '你們': {'tf': 47, 'df': 7, 'tf-idf': 6.275975453352561},
 '哈哈': {'tf': 44, 'df': 6, 'tf-idf': 12.658011187878357},
 'nd': {'tf': 44, 'df': 7, 'tf-idf': 5.875381275478993},
 'dy': {'tf': 44, 'df': 7, 'tf-idf': 5.875381275478993},
 'ndy': {'tf': 44, 'df': 7, 'tf-idf': 5.875381275478993},
 '真的': {'tf': 41, 'df': 7, 'tf-idf': 5.474787097605425},
 'ai': {'tf':

In [215]:
i = 0
term_table=defaultdict(dict)
for item in ordered_dict.items():
    item[1]['tf-idf'] = item[1]['tf']*math.log(N/item[1]['df'])
    word = list(ordered_dict)[i]
    next_word = list(ordered_dict)[i+1]
    print(strInter(word,previous_word),word,previous_word,i,i+1)
    i += 1
    if strInter(word,previous_word)/len(word) > 0.5:
        longest = next_word
    else:
        term_table[item[0]] = item[1]

#count_overlapping(OrderedDict[1],OrderedDict[0]) 

0 家寧 我愛眾量 0 1
1 量級 我愛眾量 1 2
2 眾量 我愛眾量 2 3
2 眾量級 我愛眾量 3 4
2 我愛 我愛眾量 4 5
2 愛眾 我愛眾量 5 6
3 愛眾量 我愛眾量 6 7
3 愛眾量級 我愛眾量 7 8
3 我愛眾 我愛眾量 8 9
4 我愛眾量 我愛眾量 9 10
4 我愛眾量級 我愛眾量 10 11
0 你們 我愛眾量 11 12
0 哈哈 我愛眾量 12 13
0 nd 我愛眾量 13 14
0 dy 我愛眾量 14 15
0 ndy 我愛眾量 15 16
0 真的 我愛眾量 16 17
0 ai 我愛眾量 17 18
0 ma 我愛眾量 18 19
0 il 我愛眾量 19 20
0 椅子 我愛眾量 20 21
0 哈哈哈 我愛眾量 21 22
0 mai 我愛眾量 22 23
0 ail 我愛眾量 23 24
0 mail 我愛眾量 24 25
0 om 我愛眾量 25 26
0 希望 我愛眾量 26 27
0 co 我愛眾量 27 28
0 com 我愛眾量 28 29
0 影片 我愛眾量 29 30
0 lc 我愛眾量 30 31
0 00 我愛眾量 31 32
0 ilc 我愛眾量 32 33
0 lco 我愛眾量 33 34
0 ailc 我愛眾量 34 35
0 ilco 我愛眾量 35 36
0 lcom 我愛眾量 36 37
0 mailc 我愛眾量 37 38
0 ailco 我愛眾量 38 39
0 ilcom 我愛眾量 39 40
0 mailco 我愛眾量 40 41
0 ailcom 我愛眾量 41 42
0 gm 我愛眾量 42 43
0 gma 我愛眾量 43 44
0 gmai 我愛眾量 44 45
0 gmail 我愛眾量 45 46
0 gmailc 我愛眾量 46 47
0 11 我愛眾量 47 48
0 最後 我愛眾量 48 49
0 可以 我愛眾量 49 50
1 級我 我愛眾量 50 51
0 看到 我愛眾量 51 52
2 量級我 我愛眾量 52 53
3 眾量級我 我愛眾量 53 54
1 可愛 我愛眾量 54 55
0 000 我愛眾量 55 56
4 愛眾量級我 我愛眾量 56 57
4 我愛眾量級我 我愛眾量 57 58
1 m我 我愛眾量 58 59
0 0000 我愛眾量 

IndexError: list index out of range

In [15]:
from ckip import CkipSegmenter
segmenter = CkipSegmenter()

text = '找到自己適合的床還是最重要的呢你們覺得越貴的床會越好躺嗎。6位數啊我只能躺三位數的幫我配個樂登登登登。下次希望我們開箱或測試什麼告訴我們我們來挑戰7天測試。頭媽我搶到第一了我保證我是第一。好喜歡你們穿情侶裝。我居然有點想換床墊每天都失眠阿阿。不是我吹試躺著工作我可以睡到公司破產。為什麼我覺得每一個影片我們都在做電燈泡我沒有說啊呵呵'
with open('test.txt','r',encoding = 'utf-8') as file:
    test_text = file.readline()


result = segmenter.seg(text)
# result.res is a list of tuples contain a token and its pos-tag.
#print('result.res: {}\n'.format(result.res))

# result.tok and result.pos contains only tokens and pos-tags respectively.
print('result.tok: {}\n'.format(result.tok))
#print('result.pos: {}\n'.format(result.pos))

s [748]
with open('test.txt','r',encoding = 'utf-8') as file:
    test_text = file.readline()
test_text[3271]

result.tok: ['找到', '自己', '適合', '的', '床', '還是', '最', '重要', '的', '呢', '你們', '覺得', '越', '貴', '的', '床', '會', '越', '好', '躺', '嗎', '。', '6', '位數', '啊', '我', '只', '能', '躺三', '位數', '的', '幫', '我', '配個樂', '登登', '登登', '。', '下', '次', '希望', '我們', '開箱', '或', '測試', '什麼', '告訴', '我們', '我們', '來', '挑戰', '7', '天', '測試', '。', '頭', '媽', '我', '搶到', '第', '一了', '我', '保證', '我', '是', '第一', '。', '好', '喜歡', '你們', '穿', '情侶裝', '。', '我', '居然', '有點', '想', '換', '床墊', '每', '天都', '失眠', '阿阿', '。', '不', '是', '我', '吹', '試', '躺', '著', '工作', '我', '可以', '睡到', '公司', '破產', '。', '為什麼', '我', '覺得', '每', '一', '個', '影片', '我們', '都', '在', '做', '電燈泡', '我', '沒有', '說', '啊', '呵呵']



In [224]:
from CKIP_client import ckip_client
text1 = "Facebook 是一個聯繫朋友、工作夥伴、同學或其他社交圈之間的社交工具。"
text2 = u"Facebook 是一個聯繫朋友、工作夥伴、同學或其他社交圈之間的社交工具。"
try:
    #指定輸出檔案
    ckip_client(text1,"output1.txt")
    ckip_client(text2.encode('utf-8'),"output2.txt")
    #不指定輸出檔案
    result, the_length = ckip_client(text1)
except Exception as e: print(e)

status_code: 3, Authentication failed


In [70]:
from ckipclient import CKIPClient

sample_text = '第二屆內地搖滾音樂祭將在 9 月 24 日登場！今年共有五十五組獨立樂團'\
              '參與演出，包含曾獲得金曲獎的「舒米恩」、「滅火器」等，陣容十分堅強。'

ckip = CKIPClient('140.109.19.104', 1501, 'jessica', '0909340409')
sample_results = ckip.segment(sample_text)

for sentence in sample_results:
    print('／'.join(' '.join(word) for word in sentence))

ConnectionError: ('3', 'Authentication failed')

In [33]:
youtuber_list[youtuber_list['name'].str.match('小玉')]
if '小玉' in youtuber_list['name'].values:
    print('yup')
if 'DE Jun' in data[2]['title']:
    print('yup')

yup
yup


In [338]:
dcard_path = '/Users/student/Desktop/IM final/dcard_data'
directory = os.path.join(dcard_path, 'UC8MQLq2uFUa-HXHCVjHlc-Q.json')
with open(directory) as json_file:
    dcard_data = json.load(json_file)


In [339]:
dcard_data[0]

{'id': 226464364,
 'title': '關於聖結石之類的事，我有話想說...',
 'content': '我是聖粉，我試問一句，請問他用這種方式賺錢哪裡礙到你們了??他有用你的或是你爸媽的稅金錢嗎??有嗎??你如果覺得這樣好賺不公平，你可以去嘗試啊，看你覺得好不好賺錢，不要別人用這方法過上還不錯的生活你就在那邊眼紅。\n\n你有想過他可能為了維持每天發一部片而想題材想到三更半夜嗎？大家說的沒錯，也許這會造成影片品質下降，但是他有這樣的毅力，你有嗎？我承認我沒有，他可能每天想題材想到很晚才睡，那你在幹嘛？打LOL?OW?\n\n說實在的我很看不慣一些酸民的評論，你以為光是點閱率就有收入可以拿?你想的太美好了吧，你以為這麼容易?動動那顆平時不會動的腦袋吧，這麼好做大家為什麼不去做?臺灣的酸民文化何時才會停止?\n\n大家可以不要只是一昧的批評別人嗎？可以試著去了解這份工作背後需要付出的辛苦與努力嗎？\n\n多一點鼓勵與建議，少一點批評與謾罵這世界才會進步，不是看到別人好就眼紅一直找人家毛病。\n\n我覺得每位youtuber都值得被尊敬，他們花了大把心思為的是什麼，除了賺錢之外就是希望能逗閱聽者開心啊，不是嗎？\n\n我是聖粉我驕傲，聖結石與聖嫂DoDo加油~~',
 'excerpt': '我是聖粉，我試問一句，請問他用這種方式賺錢哪裡礙到你們了？？他有用你的或是你爸媽的稅金錢嗎？？有嗎？？你如果覺得這樣好賺不公平，你可以去嘗試啊，看你覺得好不好賺錢，不要別人用這方法過上還不錯的生活你就在那邊眼紅。',
 'anonymousSchool': True,
 'anonymousDepartment': True,
 'pinned': False,
 'forumId': '75a726e6-d4e3-4902-a410-2430a39fffcb',
 'replyId': None,
 'createdAt': '2017-05-24T19:26:11.748Z',
 'updatedAt': '2017-05-25T23:25:30.388Z',
 'commentCount': 24,
 'likeCount': 63,
 'tags': ['光是',
  '這會',
  '看不慣',
  '樓上',
  '之類',
  '好做',
  '你

In [124]:
with open('feature_data/老皮.json') as json_file:
    data = json.load(json_file)

In [125]:
list(data.keys())[0: 25] 

['恐怖',
 '好笑',
 '遊戲',
 '加油',
 '嚇到',
 '封面',
 '系列',
 '恭喜',
 '這遊戲',
 '喜歡',
 '影片',
 '繼續',
 '期待',
 '一集',
 'XDD',
 '知道',
 '啊啊啊',
 '配音',
 '皮弟',
 '好像',
 '好看',
 '實況',
 '好玩',
 'XDDD',
 '結局']